In [1]:
import pandas as pd

# Leer los datos desde el archivo CSV
df_res_sin_nan = pd.read_csv('new_df_res.csv')

# Verificar los primeros registros para asegurarse de que los datos se han cargado correctamente
print(df_res_sin_nan.head())

C:\Users\danny\AppData\Local\Temp\ipykernel_5128\2928027557.py:4: DtypeWarning: Columns (700) have mixed types. Specify dtype option on import or set low_memory=False.
  df_res_sin_nan = pd.read_csv('new_df_res.csv')


   cat__subject_  cat__subject_Anal Fissure  cat__subject_Arthritis  \
0            1.0                        0.0                     0.0   
1            0.0                        0.0                     0.0   
2            1.0                        0.0                     0.0   
3            0.0                        0.0                     0.0   
4            0.0                        0.0                     0.0   

   cat__subject_Asking about Hairy cell leukemia  cat__subject_Asthma  \
0                                            0.0                  0.0   
1                                            0.0                  0.0   
2                                            0.0                  0.0   
3                                            0.0                  0.0   
4                                            0.0                  0.0   

   cat__subject_Azithromycin 250mg gluten  cat__subject_Body Lice  \
0                                     0.0                     0.0

In [2]:
import pandas as pd
import joblib

# Leer los datos desde el archivo CSV
df_res_sin_nan = pd.read_csv('new_df_res.csv')

# Verificar los primeros registros para asegurarse de que los datos se han cargado correctamente
print(df_res_sin_nan.head())

# Cargar el pipeline de preprocesamiento y el modelo entrenado
pipeline = joblib.load('imputador.pkl')
modelo = joblib.load('modelo_entrenado.pkl')

# Separar características y etiquetas si es necesario
if 'type' in df_res_sin_nan.columns:
    X_nuevos = df_res_sin_nan.drop('type', axis=1)
else:
    X_nuevos = df_res_sin_nan

# Preprocesar los nuevos datos
X_nuevos_preprocesado = pipeline.transform(X_nuevos)

# Realizar predicciones
predicciones = modelo.predict(X_nuevos_preprocesado)

# Crear un DataFrame con los resultados
resultados = df_res_sin_nan.copy()
resultados['prediccion'] = predicciones

# Guardar los resultados en un archivo CSV
resultados.to_csv('resultados_predicciones.csv', index=False)

print("Predicciones realizadas y guardadas en 'resultados_predicciones.csv'.")


C:\Users\danny\AppData\Local\Temp\ipykernel_5128\2297480094.py:5: DtypeWarning: Columns (700) have mixed types. Specify dtype option on import or set low_memory=False.
  df_res_sin_nan = pd.read_csv('new_df_res.csv')


   cat__subject_  cat__subject_Anal Fissure  cat__subject_Arthritis  \
0            1.0                        0.0                     0.0   
1            0.0                        0.0                     0.0   
2            1.0                        0.0                     0.0   
3            0.0                        0.0                     0.0   
4            0.0                        0.0                     0.0   

   cat__subject_Asking about Hairy cell leukemia  cat__subject_Asthma  \
0                                            0.0                  0.0   
1                                            0.0                  0.0   
2                                            0.0                  0.0   
3                                            0.0                  0.0   
4                                            0.0                  0.0   

   cat__subject_Azithromycin 250mg gluten  cat__subject_Body Lice  \
0                                     0.0                     0.0

In [5]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configuración de MLflow
mlflow.set_tracking_uri('http://localhost:8080')
mlflow.set_experiment('modelo_entrenado.pkl')

# Cargar datos de ejemplo
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

# Inicia una nueva ejecución en MLflow
with mlflow.start_run():
    # Parámetros del modelo
    n_estimators = 100
    max_depth = 5
    
    # Registrar parámetros
    mlflow.log_param('n_estimators', n_estimators)
    mlflow.log_param('max_depth', max_depth)
    
    # Entrena el modelo
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    # Registrar métricas
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('precision', precision)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('f1_score', f1)

    # Registrar el modelo
    mlflow.sklearn.log_model(model, 'random_forest_model')


In [6]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler

# Definir la función de preprocesamiento
def preprocess_data(data):
    # Manejo de valores faltantes
    data.fillna(data.mean(), inplace=True)
    
    # Normalización de datos
    scaler = StandardScaler()
    data_normalized = scaler.fit_transform(data)
    
    # Codificación de variables categóricas.
    data_encoded = pd.get_dummies(data, drop_first=True)
    
    # Ajustar escalador sobre los datos codificados
    data_encoded = scaler.fit_transform(data_encoded)
    
    return data_encoded

# Leer los datos desde el archivo CSV
df_res_sin_nan = pd.read_csv('new_df_res.csv')

# Verificar los primeros registros para asegurarse de que los datos se han cargado correctamente
print(df_res_sin_nan.head())

# Cargar el modelo entrenado
modelo = joblib.load('modelo_entrenado.pkl')

# Separar características y etiquetas 
if 'type' in df_res_sin_nan.columns:
    X_nuevos = df_res_sin_nan.drop('type', axis=1)
else:
    X_nuevos = df_res_sin_nan

# Preprocesar los nuevos datos utilizando la función definida
X_nuevos_preprocesado = preprocess_data(X_nuevos)

# Realizar predicciones
predicciones = modelo.predict(X_nuevos_preprocesado)

# Crear un DataFrame con los resultados
resultados = df_res_sin_nan.copy()
resultados['prediccion'] = predicciones

# Guardar los resultados en un archivo CSV
resultados.to_csv('resultados_predicciones.csv', index=False)

print("Predicciones realizadas y guardadas en 'resultados_predicciones.csv'.")


C:\Users\danny\AppData\Local\Temp\ipykernel_5128\1517283204.py:24: DtypeWarning: Columns (700) have mixed types. Specify dtype option on import or set low_memory=False.
  df_res_sin_nan = pd.read_csv('new_df_res.csv')


   cat__subject_  cat__subject_Anal Fissure  cat__subject_Arthritis  \
0            1.0                        0.0                     0.0   
1            0.0                        0.0                     0.0   
2            1.0                        0.0                     0.0   
3            0.0                        0.0                     0.0   
4            0.0                        0.0                     0.0   

   cat__subject_Asking about Hairy cell leukemia  cat__subject_Asthma  \
0                                            0.0                  0.0   
1                                            0.0                  0.0   
2                                            0.0                  0.0   
3                                            0.0                  0.0   
4                                            0.0                  0.0   

   cat__subject_Azithromycin 250mg gluten  cat__subject_Body Lice  \
0                                     0.0                     0.0

In [10]:
import csv
import json

def csv_to_json(csv_file, json_file):
    # Abre el archivo CSV en modo lectura
    with open(csv_file, 'r', newline='') as csvfile:
        # Lee los datos del archivo CSV
        csv_reader = csv.DictReader(csvfile)
        # Convierte los datos a una lista de diccionarios
        data = list(csv_reader)

    # Abre el archivo JSON en modo escritura
    with open(json_file, 'w') as jsonfile:
        # Escribe los datos en el archivo JSON
        json.dump(data, jsonfile, indent=4)

# Archivo CSV de entrada
csv_file = 'resultados_predicciones.csv'
# Archivo JSON de salida
json_file = 'resultados_predicciones.json'

# Convierte el archivo CSV a JSON
csv_to_json(csv_file, json_file)

print("Se ha convertido el archivo CSV a JSON con éxito.")


In [7]:
import mlflow
import mlflow.sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configura la URI de seguimiento de MLflow (local o remoto)
mlflow.set_tracking_uri("http://localhost:8080")  # Ejemplo con servidor local
mlflow.set_experiment("modelo_entrenado.pkl")

def log_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    with mlflow.start_run():
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

    return accuracy, precision, recall, f1
